# Лабораторная работа №5. Применение сверточных нейронных сетей (бинарная классификация)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

## Задание 1.
Загрузите данные. Разделите исходный набор данных на обучающую, валидационную и контрольную выборки.

In [4]:
# Скачиваем данные ручками и ложим в data директорию, где хранятся датасеты из всех лабораторных.
import os
import pandas as pd

dataset_path = os.path.join('data', 'dogs-vs-cats')

data = [[os.path.basename(filename), 'dog' if 'dog' in filename else 'cat']
        for filename in os.listdir(os.path.join(dataset_path, 'train'))]

data_df = pd.DataFrame(data=data, columns=['filename', 'label'])

In [5]:
from sklearn.model_selection import train_test_split
train_df, validate_df = train_test_split(data_df, test_size=0.2, random_state=42)
validate_df, test_df = train_test_split(validate_df, test_size=0.2, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.shape, validate_df.shape, test_df.shape

((20000, 2), (4000, 2), (1000, 2))

## Задание 2.
Реализуйте глубокую нейронную сеть с как минимум тремя сверточными слоями. Какое качество классификации получено?

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, BatchNormalization, Dropout, Flatten


network = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


network.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])    
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 62, 62, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_data_generator(df):
    img_h, img_w = 128, 128
    gen = ImageDataGenerator(rescale=1./ 255)
    return gen.flow_from_dataframe(
        df, 
        'data/dogs-vs-cats/train/', 
        x_col='filename',
        y_col='label',
        target_size=(img_w, img_h),
        class_mode='binary',
        batch_size=128
    )

train_gen = get_data_generator(train_df)
valid_gen = get_data_generator(validate_df)
test_gen = get_data_generator(test_df)

Found 20000 validated image filenames belonging to 2 classes.
Found 4000 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.


In [6]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [9]:
network.fit(
    train_gen, 
    epochs=15,
    validation_data=valid_gen,
    workers=4
)

Train for 157 steps, validate for 32 steps
Epoch 1/15
157/157 [==============================] - 51s 322ms/step - loss: 0.0263 - accuracy: 0.9909 - val_loss: 0.6458 - val_accuracy: 0.8605
Epoch 2/15
157/157 [==============================] - 50s 321ms/step - loss: 0.0215 - accuracy: 0.9929 - val_loss: 0.6492 - val_accuracy: 0.8490
Epoch 3/15
157/157 [==============================] - 50s 321ms/step - loss: 0.0226 - accuracy: 0.9918 - val_loss: 0.5901 - val_accuracy: 0.8695
Epoch 4/15
157/157 [==============================] - 50s 321ms/step - loss: 0.0234 - accuracy: 0.9916 - val_loss: 0.6021 - val_accuracy: 0.8683
Epoch 5/15
157/157 [==============================] - 50s 321ms/step - loss: 0.0254 - accuracy: 0.9902 - val_loss: 0.5575 - val_accuracy: 0.8767
Epoch 6/15
157/157 [==============================] - 50s 321ms/step - loss: 0.0252 - accuracy: 0.9913 - val_loss: 0.6220 - val_accuracy: 0.8593
Epoch 7/15
157/157 [==============================] - 50s 322ms/step - loss: 0.0268 - a

In [10]:
network.evaluate(test_gen)

8/8 [==============================] - 2s 224ms/step - loss: 0.6267 - accuracy: 0.8560


[0.626665111631155, 0.856]

## Задание 3.
Примените дополнение данных (data augmentation). Как это повлияло на качество классификатора?

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def get_augmented_data_generator(df):
    img_h, img_w = 128, 128

    gen = ImageDataGenerator(
        rotation_range=15,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        width_shift_range=0.1,
        height_shift_range=0.1
    )
    
    return gen.flow_from_dataframe(
        df, 
        'data/dogs-vs-cats/train/', 
        x_col='filename',
        y_col='label',
        target_size=(img_w, img_h),
        class_mode='binary',
        batch_size=64
    )


train_augmented_gen = get_augmented_data_generator(train_df)
valid_augmented_gen = get_augmented_data_generator(validate_df)
test_augmented_gen = get_augmented_data_generator(test_df)

Found 20000 validated image filenames belonging to 2 classes.
Found 4000 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.


In [7]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, BatchNormalization, Dropout, Flatten


network = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


network.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])    
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
batch_normalization_5 (Batch (None, 62, 62, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 64)       

In [12]:
network.fit(
    train_augmented_gen,
    epochs=50,
    validation_data=valid_augmented_gen,
    workers=4
)

Train for 313 steps, validate for 63 steps
Epoch 1/50
313/313 [==============================] - 98s 313ms/step - loss: 0.7422 - accuracy: 0.6355 - val_loss: 0.8462 - val_accuracy: 0.5060
Epoch 2/50
313/313 [==============================] - 97s 310ms/step - loss: 0.5807 - accuracy: 0.7059 - val_loss: 0.5608 - val_accuracy: 0.7135
Epoch 3/50
313/313 [==============================] - 97s 311ms/step - loss: 0.5156 - accuracy: 0.7459 - val_loss: 0.4792 - val_accuracy: 0.7715
Epoch 4/50
313/313 [==============================] - 97s 310ms/step - loss: 0.4706 - accuracy: 0.7729 - val_loss: 0.5134 - val_accuracy: 0.7477
Epoch 5/50
313/313 [==============================] - 97s 310ms/step - loss: 0.4390 - accuracy: 0.7952 - val_loss: 0.4884 - val_accuracy: 0.7645
Epoch 6/50
313/313 [==============================] - 97s 311ms/step - loss: 0.4158 - accuracy: 0.8102 - val_loss: 0.4993 - val_accuracy: 0.7665
Epoch 7/50
313/313 [==============================] - 97s 310ms/step - loss: 0.3895 - a

In [13]:
network.evaluate(test_augmented_gen)

16/16 [==============================] - 4s 237ms/step - loss: 0.3057 - accuracy: 0.9070


[0.3057435564696789, 0.907]

In [16]:
network.evaluate(test_gen)

8/8 [==============================] - 2s 256ms/step - loss: 0.2618 - accuracy: 0.9100


[0.26183051988482475, 0.91]

#### Как видим точность выше примерно на 6 процентов на одной и той же контрольной выборке.

## Задание 4.
Поэкспериментируйте с готовыми нейронными сетями (например, AlexNet, VGG16, Inception и т.п.), применив передаточное обучение. Как это повлияло на качество классификатора?

In [21]:
# В качестве готовой сети возьмем VGG19
# https://keras.io/applications/#vgg19

from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

image_size = 224
input_shape = (224, 224, 3)

pre_trained_model = VGG19(input_shape=input_shape, include_top=False, weights="imagenet")
    
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False

for layer in pre_trained_model.layers[15:]:
    layer.trainable = True
    
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
    
x = GlobalAveragePooling2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

network = Model(pre_trained_model.input, x)

network.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

network.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [22]:
def get_data_generator(df):
    gen = ImageDataGenerator(
        rotation_range=15,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        width_shift_range=0.1,
        height_shift_range=0.1
    )
    
    return gen.flow_from_dataframe(
        df, 
        'data/dogs-vs-cats/train/', 
        x_col='filename',
        y_col='category',
        target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
        class_mode='binary',
        batch_size=32
    )


train_augmented_gen = get_augmented_data_generator(train_df)
valid_augmented_gen = get_augmented_data_generator(validate_df)

Found 20000 validated image filenames belonging to 2 classes.
Found 4000 validated image filenames belonging to 2 classes.


In [26]:
network.fit(
    train_augmented_gen, 
    epochs=20,
    validation_data=valid_augmented_gen,
    workers=4,
)

Train for 313 steps, validate for 63 steps
Epoch 1/20
313/313 [==============================] - 124s 397ms/step - loss: 0.5481 - accuracy: 0.7224 - val_loss: 0.4328 - val_accuracy: 0.8048
Epoch 2/20
313/313 [==============================] - 125s 398ms/step - loss: 0.3914 - accuracy: 0.8300 - val_loss: 0.3425 - val_accuracy: 0.8490
Epoch 3/20
313/313 [==============================] - 125s 398ms/step - loss: 0.3159 - accuracy: 0.8660 - val_loss: 0.2867 - val_accuracy: 0.8742
Epoch 4/20
313/313 [==============================] - 124s 398ms/step - loss: 0.2761 - accuracy: 0.8849 - val_loss: 0.2578 - val_accuracy: 0.8945
Epoch 5/20
313/313 [==============================] - 124s 396ms/step - loss: 0.2545 - accuracy: 0.8955 - val_loss: 0.2761 - val_accuracy: 0.8790
Epoch 6/20
313/313 [==============================] - 124s 397ms/step - loss: 0.2415 - accuracy: 0.8999 - val_loss: 0.2190 - val_accuracy: 0.9085
Epoch 7/20
313/313 [==============================] - 124s 397ms/step - loss: 0.2

In [27]:
network.evaluate(test_gen)

8/8 [==============================] - 5s 654ms/step - loss: 0.1575 - accuracy: 0.9410


[0.157472581602633, 0.941]

#### Как видим точность еще выше примерно на 3 процента на одной и той же контрольной выборке.